In [ ]:
%pip install pandas seaborn matplotlib

In [ ]:
import pandas as pd

JOIN_KEYS = ["año", "capitulo", "nombre_capitulo"]

presupuesto_df = (
    pd.read_csv("../datasets/pagos_salud_chile_2018-2024.csv", delimiter=",")
    .rename(columns={"devengo": "total"})
    .groupby(["año", "capitulo", "nombre_capitulo", "subtitulo", "nombre_subtitulo"])[
        ["total"]
    ]
    .agg("sum")
    .reset_index()
)

subtitulo_columns = (
    presupuesto_df[["subtitulo", "nombre_subtitulo"]]
    .sort_values(by="subtitulo")
    .drop_duplicates()["nombre_subtitulo"]
    .to_list()
)

presupuesto_df = (
    presupuesto_df.pivot_table(
        index=["año", "capitulo", "nombre_capitulo"],
        columns=["nombre_subtitulo"],
        values=["total"],
    )
    .fillna(0)
    .reset_index()
)

presupuesto_df = pd.concat(
    [
        presupuesto_df[["año", "capitulo", "nombre_capitulo"]],
        presupuesto_df["total"][subtitulo_columns],
    ],
    axis=1,
)
presupuesto_df.columns = [
    "año",
    "capitulo",
    "nombre_capitulo",
] + presupuesto_df.columns[3:].to_list()
presupuesto_df.columns = presupuesto_df.columns.str.lower().str.replace(" ", "_")
presupuesto_df["total_gasto"] = presupuesto_df.iloc[:, 3:].sum(axis=1)

presupuesto_df

In [ ]:
YEARS = range(2018, 2024)

intento_ideacion_df = pd.read_csv("../datasets/ideacion_intento.csv", delimiter=",")
urgencia_autolesiones_df = pd.read_csv(
    "../datasets/urgencia_lesiones_autoinflingidas.csv", delimiter=","
)
casen_df = pd.read_csv("../datasets/casen.csv", delimiter=",")
egresos_df = pd.read_csv("../datasets/egresos_hospitalarios.csv", delimiter=",")

joined_df = pd.merge(presupuesto_df, intento_ideacion_df, on=JOIN_KEYS, how="outer")
joined_df = pd.merge(joined_df, urgencia_autolesiones_df, on=JOIN_KEYS, how="outer")
joined_df = pd.merge(joined_df, casen_df, on=JOIN_KEYS, how="outer")
joined_df = pd.merge(joined_df, egresos_df, on=JOIN_KEYS, how="outer")
joined_df = joined_df.query("año >= 2018 and año <= 2023").reset_index(drop=True)
sorted(egresos_df["nombre_capitulo"].unique())

In [ ]:
joined_df.info()

In [ ]:
# joined_df.filter(like="hospitalizaciones").columns.to_list()

In [ ]:
# joined_df.filter(like="hospitalizacion_total").columns.to_list()

In [ ]:
joined_df[
    ["año", "capitulo", "nombre_capitulo"]
    + joined_df.filter(like="promedio").columns.to_list()
]

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["suicidio_ideacion"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["suicidio_intento"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["urgencias_lesiones_autoinflingidas"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
for year in YEARS:
    print(
        year,
        pd.cut(
            joined_df[joined_df["año"] == year]["suicidio_intento"], 8
        ).value_counts(),
        "\n",
    )

In [ ]:
joined_df.query("año == 2023")["urgencias_lesiones_autoinflingidas"].plot.hist(
    title="Urgencias por Lesiones Autoinflingidas - Año 2023", figsize=(6, 6)
)

In [ ]:
joined_df.query("año == 2018")["nombre_capitulo"].value_counts()

In [ ]:
def get_top_k_columns_with_less_zeroes(k, pattern, df):
    promedio_variables = df.filter(like=pattern).columns.to_list()

    zero_counts = df[promedio_variables].apply(lambda col: (col == 0).sum())
    top_k = zero_counts.sort_values(ascending=True).index[:k]
    return top_k.to_list()


get_top_k_columns_with_less_zeroes(10, "promedio", joined_df)

In [ ]:
get_top_k_columns_with_less_zeroes(10, "promedio", joined_df)

In [ ]:
joined_df[
    "top_5_dias_hospitalizacion_promedio_esquizofrenia_trastornos_esquizotípicos_y_trastornos_delirantes"
].hist()

In [ ]:
joined_df.to_csv("../datasets/eda.csv", index=False)

In [ ]:
joined_df["top_5_dias_hospitalizacion_total_diabetes_mellitus"].hist()

In [ ]:
variable_diabetes = "top_5_dias_hospitalizacion_total_diabetes_mellitus"

joined_df[joined_df[variable_diabetes] != 0][variable_diabetes].hist()

In [ ]:
joined_df

In [ ]:
joined_df.filter(like="desarrollo").columns

In [ ]:
clustering_columns = [
    "gastos_en_personal",
    "bienes_y_servicios_de_consumo",
    "prestaciones_de_seguridad_social",
    "iniciativas_de_inversion",
    "transferencias_corrientes",
    "adquisicion_de_activos_no_financieros",
    "total_gasto",
    "total_fonasa",
    "suicidio_ideacion",
    "suicidio_intento",
    "urgencias_lesiones_autoinflingidas",
    "problemas_ser_atendido_si",
    "problemas_ser_atendido_no",
    "problemas_conseguir_cita_si",
    "problemas_conseguir_cita_no",
    "problemas_llegar_consulta_si",
    "problemas_llegar_consulta_no",
    "problemas_pagar_por_costo_si",
    "problemas_pagar_por_costo_no",
    "top_5_cantidad_hospitalizaciones_asignación_provisoria_de_nuevas_afecciones_de_etiología_incierta",
    "top_5_dias_hospitalizacion_total_asignación_provisoria_de_nuevas_afecciones_de_etiología_incierta",
    "top_5_cantidad_hospitalizaciones_atención_materna_relacionada_con_el_feto_y_la_cavidad_amniótica_y_con_posibles_problemas_del_parto_",
    "top_5_cantidad_hospitalizaciones_complicaciones_del_trabajo_de_parto_y_del_parto",
    "top_5_cantidad_hospitalizaciones_parto",
    "top_5_dias_hospitalizacion_total_parto",
    "top_5_dias_hospitalizacion_promedio_trastornos_del_sistema_digestivo_del_feto_y_del_recién_nacido",
    "top_5_cantidad_hospitalizaciones_enfermedades_cerebrovasculares",
    "top_5_cantidad_hospitalizaciones_enfermedades_del_apéndice",
    "top_5_cantidad_hospitalizaciones_influenza_[gripe]_y_neumonía",
    "top_5_dias_hospitalizacion_total_influenza_[gripe]_y_neumonía",
    "top_5_cantidad_hospitalizaciones_otras_formas_de_enfermedad_del_corazón",
    "top_5_dias_hospitalizacion_total_otras_formas_de_enfermedad_del_corazón",
    "top_5_dias_hospitalizacion_total_enfermedades_isquémicas_del_corazón",
    "top_5_cantidad_hospitalizaciones_trastornos_de_la_vesícula_biliar_de_las_vías_biliares_y_del_páncreas",
    "top_5_dias_hospitalizacion_total_trastornos_de_la_vesícula_biliar_de_las_vías_biliares_y_del_páncreas",
    "top_5_dias_hospitalizacion_promedio_esquizofrenia_trastornos_esquizotípicos_y_trastornos_delirantes",
    "top_5_dias_hospitalizacion_total_esquizofrenia_trastornos_esquizotípicos_y_trastornos_delirantes",
    "top_5_dias_hospitalizacion_promedio_trastornos_del_desarrollo_psicológico",
    "top_5_dias_hospitalizacion_promedio_retraso_mental",
    "top_5_dias_hospitalizacion_promedio_trastornos_mentales_orgánicos_incluidos_los_trastornos_sintomáticos",
    "top_5_dias_hospitalizacion_promedio_malformaciones_congénitas_del_sistema_respiratorio",
    "top_5_dias_hospitalizacion_promedio_otras_enfermedades_degenerativas_del_sistema_nervioso",
    "top_5_dias_hospitalizacion_promedio_personas_con_riesgos_potenciales_para_su_salud_relacionados_con_circunstancias_socioeconómicas_y_psicosociales",
    "top_5_dias_hospitalizacion_promedio_tuberculosis",
    "top_5_dias_hospitalizacion_total_complicaciones_de_la_atención_médica_y_quirúrgica_no_clasificadas_en_otra_parte",
    "top_5_dias_hospitalizacion_total_diabetes_mellitus",
]

joined_df[["año", "capitulo", "nombre_capitulo"] + clustering_columns].to_csv(
    "../datasets/analisis_presupuestario_salud.csv", index=False
)